---
title: Software Review Timelines
subtitle: pyOpenSci Peer Review Summary Stats
license:
  code: BSD-3-Clause
---

This is a workflow that colates all GitHub issues associated with our reviews. 

In [4]:
# https://github.com/ryantam626/jupyterlab_code_formatter
import warnings
from datetime import datetime, timezone
from pathlib import Path

import altair as alt
import pandas as pd
import pytz

from pyosmetrics.functions import count_edits_by_quarter
from pyosmetrics.plot_theme import load_poppins_font, register_and_enable_poppins_theme


# Suppress all warnings
warnings.filterwarnings("ignore")
# Load & register Poppins font and theme for the page
load_poppins_font()
register_and_enable_poppins_theme()

In [7]:
# Get review data from .csv file
current_dir = Path.cwd()
parent_dir = current_dir.parent.parent
data_dir = parent_dir / "_data"
file_path = data_dir / "review_submissions.csv"
reviews = pd.read_csv(file_path, parse_dates=["date_opened", "date_closed"])
total_submissions = len(reviews)

In [8]:
# calculate total time open for each review
closed = reviews.dropna(subset=['date_closed'])
closed['days_open'] = (closed['date_closed'] - closed['date_opened']).dt.days

closed['quarter'] = closed['date_opened'].dt.to_period('Q')
closed.head()
summary_by_quarter = closed.groupby('quarter').agg({'days_open': 'mean', 'package_name': 'count'})
summary_by_quarter = summary_by_quarter.rename(columns={'package_name': 'count'})
summary_by_quarter = summary_by_quarter.reset_index() 

In [11]:
# Altair doesn't support pandas periods, convert to str
summary_by_quarter['quarter'] = summary_by_quarter['quarter'].astype(str)
# Create the Altair chart using object-oriented syntax
chart = (
    alt.Chart(summary_by_quarter)
    .mark_line(point=alt.MarkConfig(filled=True, size=100), strokeWidth=1)
    .encode(
        x=alt.X('quarter:O', axis=alt.Axis(labelAngle=-45)), # Ordinal quarter on the x-axis
        y=alt.Y('days_open:Q',axis=alt.Axis(format="d", tickMinStep=1) ),  # Quantitative days_open on the y-axis
        tooltip=['days_open']
    )
    .properties(title="Mean Review Open Duration (days)", width="container")
    .interactive()
)

chart

alt.Chart(...)

In [22]:
# Get a list of reviews submitted to us
# This potentially doesn't include issues that were deemed out of scope...

open_reviews = reviews[reviews["date_closed"].isna()]

today = datetime.now(timezone.utc)
open_reviews["days_open"] = (today - open_reviews["date_opened"]).dt.days
open_reviews.drop(columns=["date_closed"], inplace=True)
total_open = len(open_reviews)

Unnamed: 0                    int64
package_name                 object
date_opened     datetime64[ns, UTC]
labels                       object
status                       object
days_open                     int64
dtype: object

In [33]:
# The plot below isn't super useful .
# open_reviews.head()

,Unnamed: 0,package_name,date_opened,labels,status,days_open
0,0,FElupe,2024-09-14 22:37:44+00:00,"['0/pre-review-checks', 'New Submission!']",pre-review,11
1,1,Solar Data Tools,2024-08-17 06:30:02+00:00,['2/seeking-reviewers'],under-review,39
2,2,THzTools,2024-08-01 11:54:53+00:00,['0/seeking-editor'],seeking editor,55
3,3,BlackMarblePy,2024-07-18 14:15:11+00:00,['1/editor-assigned'],under-review,69
4,4,MontePy,2024-07-01 21:04:05+00:00,['3/reviewers-assigned'],under-review,86


In [39]:

# Create the Altair chart using object-oriented syntax
chart = (
    alt.Chart(open_reviews)
    .mark_line(point=alt.MarkConfig(filled=True, size=100), strokeWidth=0)
    .encode(
        x=alt.X('date_opened:T', axis=alt.Axis(labelAngle=-45, format='%Y-%m-%d')), # Ordinal quarter on the x-axis
        y=alt.Y('days_open:Q'),  # Quantitative days_open on the y-axis
        tooltip=['package_name', 'status', 'days_open', 'date_opened']
    )
    .properties(title="Open Reviews - Days Open", width="container")
    .interactive()
)

chart

alt.Chart(...)